In [1]:
import sys
import os
# Append the library path to PYTHONPATH, so library can be imported.
sys.path.append(os.path.dirname(os.getcwd()))
import datetime
import shutil

import numpy as np
import pandas as pd

from library import network as nw
from library import common as cm

Using TensorFlow backend.


In [2]:
%run setup.py
%load_ext autoreload
%autoreload 2

np.random.seed(RANDOM_SEED)

In [3]:
%run Load_Clean_aux.py
max_period = max([int(s[6:]) for s in df.columns if 'period' in s])

C:\Users\Weiguan\Anaconda3\envs\tfenv\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


We remove samples when the next trade is not available. 163472 samples (4.52%) are removed. We have 95.48% of original data left, yielding a size of 3453492.
We remove unreasonable implvol0. 3159 samples (0.09%) are removed. We have 95.39% of original data left, yielding a size of 3450333.
We remove samples that have time to maturity less than 0 day. 0 samples (0.00%) are removed. We have 95.39% of original data left, yielding a size of 3450333.
We remove in-the-money samples. 712998 samples (20.66%) are removed. We have 75.68% of original data left, yielding a size of 2737335.
We shrink moneyness range. 104752 samples (3.83%) are removed. We have 72.78% of original data left, yielding a size of 2632583.


##### Choose features

In [4]:
if FEATURE_SET == 'normal_feature':
    ori_fea = ['M0', 'tau0_implvol0']
    sub_res = res_dir + 'Network/Normal_Feature/'

if FEATURE_SET == 'delta_vega':
    ori_fea = ['delta_bs', '1_over_sqrt_tau', 'vega_n']
    sub_res = res_dir + 'Network/Delta_Vega/'
    
if VIX:
    ori_fea += ['fake_vix']

use_fea = [x + '_t' for x in ori_fea] + ['cp_int']

os.makedirs(sub_res, exist_ok=True)
shutil.copy(f'setup.py', sub_res)
print(f'Result PNL is stored at:\n{res_dir}')

Result PNL is stored at:
C:\Users\Weiguan\Dropbox\Research\DeepHedging\Debug_0404\OptionMetrics\Result/FREQ=1D_HALFMONEY=otm_MINM=0.8_MAXM=1.5_MINTAU=0_Permute=True_VIX=False_MISSING=replace_missing_WINDOW=180D_KickTmrZeroVolume=False/


##### Setup hypers and target
Here, we specify hyperparameters, which will be used for training at the next step.

In [5]:
hypers = {
    'nodes_per_layer': (30, 30),
    'reg_alpha': 1e-4,
    'lr': 1e-4,
    'epochs': 300,
    'outact': 'linear'
}

##### Train a network with given hypers

In [6]:
if not PERMUTE:
    """
    Here, we use a bit different structure as the simulation data.
    In case of no permutation, there are only one ckp, history, plot folder.
    In case of permuation, in each of ckp, history and plot, we have many permutation folders.

    As for the pnl folder:
    In case of no permutation, only one pnl file.
    in case of permutations, many pnl files.
    This is consistent as the regression folder.
    """
    sub_res_dir = {
        'ckp': sub_res + 'ckp/',
        'history': sub_res + 'history/',
        'pnl': sub_res + 'pnl/',
        'plot': sub_res + 'plot/'
    }
    for key, value in sub_res_dir.items():
        os.makedirs(value, exist_ok=True)

    df = nw.rolling_net(
        df,
        ori_fea=ori_fea,
        use_fea=use_fea,
        end_period=max_period,
        hypers=hypers,
        sub_res_dir=sub_res_dir
    )

    cm.store_pnl(df, df['delta_nn'], sub_res_dir['pnl'] + 'pnl.csv')
    

In [ ]:
if PERMUTE:
    for p in range(NUM_PERMUTE):
        """
        We keep all the pnl in the same folder, because each
        permute only has one pnl files and we can use the same code 
        for evaluation.
        """
        sub_res_dir = {
            'ckp': sub_res + f'ckp/permute{p}/',
            'history': sub_res + f'history/permute{p}/',
            'pnl': sub_res + f'pnl/',
            'plot': sub_res + f'plot/permute{p}/'
        }
        for key, value in sub_res_dir.items():
            os.makedirs(value, exist_ok=True)
            
        df_permute = df.copy()
        df_permute = cm.rolling_permute(df_permute, random_seed=p+100)
        
        df_permute = nw.rolling_net(
            df_permute, 
            ori_fea=ori_fea, 
            use_fea=use_fea,
            end_period=max_period,
            hypers=hypers, 
            sub_res_dir=sub_res_dir)
        
        cm.store_pnl(df_permute, df_permute['delta_nn'], sub_res_dir['pnl'] + f'pnl{p}.csv')



 Working on period 0.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 147081 samples, validate on 39802 samples
Epoch 1/300
147081/147081 [==============================] - 3s 24us/step - loss: 0.0702 - mean_squared_error: 0.0667 - val_loss: 0.0139 - val_mean_squared_error: 0.0105
Epoch 2/300
147081/147081 [==============================] - 6s 43us/step - loss: 0.0137 - mean_squared_error: 0.0103 - val_loss: 0.0135 - val_mean_squared_error: 0.0103
Epoch 3/300
147081/147081 [==============================] - 5s 32us/step - loss: 0.0134 - mean_squared_error: 0.0102 - val_loss: 0.0133 - val_mean_squared_error: 0.0102
Epoch 4/300
147081/147081 [==============================] - 4s 29us/step - loss: 0.0131 - mean_squared_error: 0.0101 - val_loss: 0.0132 - val_mean_squared_error: 0.0102
Epoch 5/300
147081/147081 [==============================] - 4s 24us/step - loss: 0.0130 - mean_squared_error: 0.0101 - val_

147081/147081 [==============================] - 4s 25us/step - loss: 0.0111 - mean_squared_error: 0.0099 - val_loss: 0.0112 - val_mean_squared_error: 0.0100
Epoch 46/300
147081/147081 [==============================] - 3s 22us/step - loss: 0.0111 - mean_squared_error: 0.0099 - val_loss: 0.0112 - val_mean_squared_error: 0.0101
Epoch 47/300
147081/147081 [==============================] - 3s 21us/step - loss: 0.0111 - mean_squared_error: 0.0099 - val_loss: 0.0112 - val_mean_squared_error: 0.0100
Epoch 48/300
147081/147081 [==============================] - 3s 22us/step - loss: 0.0111 - mean_squared_error: 0.0099 - val_loss: 0.0111 - val_mean_squared_error: 0.0099
Epoch 49/300
147081/147081 [==============================] - 3s 21us/step - loss: 0.0110 - mean_squared_error: 0.0099 - val_loss: 0.0111 - val_mean_squared_error: 0.0100
Epoch 50/300
147081/147081 [==============================] - 3s 21us/step - loss: 0.0110 - mean_squared_error: 0.0099 - val_loss: 0.0110 - val_mean_squared_e

Epoch 93/300
147081/147081 [==============================] - 3s 18us/step - loss: 0.0106 - mean_squared_error: 0.0099 - val_loss: 0.0105 - val_mean_squared_error: 0.0099
Epoch 94/300
147081/147081 [==============================] - 3s 18us/step - loss: 0.0106 - mean_squared_error: 0.0099 - val_loss: 0.0106 - val_mean_squared_error: 0.0100
Epoch 95/300
147081/147081 [==============================] - 3s 18us/step - loss: 0.0106 - mean_squared_error: 0.0099 - val_loss: 0.0107 - val_mean_squared_error: 0.0100
Epoch 96/300
147081/147081 [==============================] - 3s 18us/step - loss: 0.0106 - mean_squared_error: 0.0099 - val_loss: 0.0105 - val_mean_squared_error: 0.0099
Epoch 97/300
147081/147081 [==============================] - 3s 18us/step - loss: 0.0105 - mean_squared_error: 0.0099 - val_loss: 0.0106 - val_mean_squared_error: 0.0099
Epoch 98/300
147081/147081 [==============================] - 3s 18us/step - loss: 0.0106 - mean_squared_error: 0.0099 - val_loss: 0.0105 - val_m

Epoch 141/300
147081/147081 [==============================] - 4s 26us/step - loss: 0.0104 - mean_squared_error: 0.0099 - val_loss: 0.0103 - val_mean_squared_error: 0.0099
Epoch 142/300
147081/147081 [==============================] - 4s 27us/step - loss: 0.0104 - mean_squared_error: 0.0099 - val_loss: 0.0104 - val_mean_squared_error: 0.0099
Epoch 143/300
147081/147081 [==============================] - 4s 26us/step - loss: 0.0103 - mean_squared_error: 0.0099 - val_loss: 0.0104 - val_mean_squared_error: 0.0099
Epoch 144/300
147081/147081 [==============================] - 4s 26us/step - loss: 0.0103 - mean_squared_error: 0.0099 - val_loss: 0.0105 - val_mean_squared_error: 0.0100
Epoch 145/300
147081/147081 [==============================] - 4s 27us/step - loss: 0.0103 - mean_squared_error: 0.0099 - val_loss: 0.0103 - val_mean_squared_error: 0.0099
Epoch 146/300
147081/147081 [==============================] - 4s 24us/step - loss: 0.0103 - mean_squared_error: 0.0099 - val_loss: 0.0105 -

147081/147081 [==============================] - 4s 25us/step - loss: 0.0102 - mean_squared_error: 0.0099 - val_loss: 0.0103 - val_mean_squared_error: 0.0099
Epoch 189/300
147081/147081 [==============================] - 4s 25us/step - loss: 0.0102 - mean_squared_error: 0.0099 - val_loss: 0.0102 - val_mean_squared_error: 0.0098
Epoch 190/300
147081/147081 [==============================] - 4s 25us/step - loss: 0.0102 - mean_squared_error: 0.0099 - val_loss: 0.0103 - val_mean_squared_error: 0.0100
Epoch 191/300
147081/147081 [==============================] - 4s 25us/step - loss: 0.0102 - mean_squared_error: 0.0099 - val_loss: 0.0102 - val_mean_squared_error: 0.0099
Epoch 192/300
147081/147081 [==============================] - 4s 27us/step - loss: 0.0102 - mean_squared_error: 0.0099 - val_loss: 0.0105 - val_mean_squared_error: 0.0102
Epoch 193/300
147081/147081 [==============================] - 4s 26us/step - loss: 0.0102 - mean_squared_error: 0.0099 - val_loss: 0.0102 - val_mean_squa

...

798847/798847 [==============================] - 19s 24us/step - loss: 0.0056 - mean_squared_error: 0.0055 - val_loss: 0.0054 - val_mean_squared_error: 0.0054
Epoch 282/300
798847/798847 [==============================] - 19s 24us/step - loss: 0.0056 - mean_squared_error: 0.0055 - val_loss: 0.0054 - val_mean_squared_error: 0.0054
Epoch 283/300
798847/798847 [==============================] - 19s 24us/step - loss: 0.0056 - mean_squared_error: 0.0055 - val_loss: 0.0055 - val_mean_squared_error: 0.0054
Epoch 284/300
798847/798847 [==============================] - 19s 24us/step - loss: 0.0056 - mean_squared_error: 0.0055 - val_loss: 0.0054 - val_mean_squared_error: 0.0054
Epoch 285/300
798847/798847 [==============================] - 20s 24us/step - loss: 0.0056 - mean_squared_error: 0.0055 - val_loss: 0.0056 - val_mean_squared_error: 0.0055
Epoch 286/300
798847/798847 [==============================] - 19s 24us/step - loss: 0.0056 - mean_squared_error: 0.0055 - val_loss: 0.0054 - val_mea

929971/929971 [==============================] - 22s 23us/step - loss: 0.0063 - mean_squared_error: 0.0061 - val_loss: 0.0061 - val_mean_squared_error: 0.0059
Epoch 29/300
929971/929971 [==============================] - 23s 25us/step - loss: 0.0063 - mean_squared_error: 0.0061 - val_loss: 0.0062 - val_mean_squared_error: 0.0059
Epoch 30/300
929971/929971 [==============================] - 22s 24us/step - loss: 0.0063 - mean_squared_error: 0.0061 - val_loss: 0.0061 - val_mean_squared_error: 0.0059
Epoch 31/300
929971/929971 [==============================] - 22s 23us/step - loss: 0.0063 - mean_squared_error: 0.0061 - val_loss: 0.0061 - val_mean_squared_error: 0.0059
Epoch 32/300
929971/929971 [==============================] - 22s 23us/step - loss: 0.0063 - mean_squared_error: 0.0061 - val_loss: 0.0061 - val_mean_squared_error: 0.0059
Epoch 33/300
929971/929971 [==============================] - 22s 23us/step - loss: 0.0063 - mean_squared_error: 0.0061 - val_loss: 0.0061 - val_mean_squ

Epoch 76/300
929971/929971 [==============================] - 24s 26us/step - loss: 0.0062 - mean_squared_error: 0.0060 - val_loss: 0.0061 - val_mean_squared_error: 0.0060
Epoch 77/300
929971/929971 [==============================] - 26s 28us/step - loss: 0.0062 - mean_squared_error: 0.0060 - val_loss: 0.0060 - val_mean_squared_error: 0.0059
Epoch 78/300
929971/929971 [==============================] - 26s 28us/step - loss: 0.0062 - mean_squared_error: 0.0060 - val_loss: 0.0060 - val_mean_squared_error: 0.0059
Epoch 79/300
929971/929971 [==============================] - 31s 33us/step - loss: 0.0062 - mean_squared_error: 0.0060 - val_loss: 0.0060 - val_mean_squared_error: 0.0059
Epoch 80/300
929971/929971 [==============================] - 29s 31us/step - loss: 0.0062 - mean_squared_error: 0.0060 - val_loss: 0.0061 - val_mean_squared_error: 0.0059
Epoch 81/300
929971/929971 [==============================] - 27s 29us/step - loss: 0.0062 - mean_squared_error: 0.0060 - val_loss: 0.0060 -

929971/929971 [==============================] - 23s 25us/step - loss: 0.0061 - mean_squared_error: 0.0060 - val_loss: 0.0060 - val_mean_squared_error: 0.0059
Epoch 124/300
929971/929971 [==============================] - 23s 25us/step - loss: 0.0061 - mean_squared_error: 0.0060 - val_loss: 0.0060 - val_mean_squared_error: 0.0059
Epoch 125/300
929971/929971 [==============================] - 23s 25us/step - loss: 0.0061 - mean_squared_error: 0.0060 - val_loss: 0.0060 - val_mean_squared_error: 0.0059
Epoch 126/300
929971/929971 [==============================] - 22s 24us/step - loss: 0.0061 - mean_squared_error: 0.0060 - val_loss: 0.0060 - val_mean_squared_error: 0.0059
Epoch 127/300
929971/929971 [==============================] - 23s 24us/step - loss: 0.0061 - mean_squared_error: 0.0060 - val_loss: 0.0061 - val_mean_squared_error: 0.0060
Epoch 128/300
929971/929971 [==============================] - 23s 25us/step - loss: 0.0061 - mean_squared_error: 0.0060 - val_loss: 0.0060 - val_mea

929971/929971 [==============================] - 24s 26us/step - loss: 0.0061 - mean_squared_error: 0.0060 - val_loss: 0.0060 - val_mean_squared_error: 0.0059
Epoch 171/300
929971/929971 [==============================] - 24s 26us/step - loss: 0.0061 - mean_squared_error: 0.0060 - val_loss: 0.0060 - val_mean_squared_error: 0.0059
Epoch 172/300
929971/929971 [==============================] - 24s 26us/step - loss: 0.0061 - mean_squared_error: 0.0060 - val_loss: 0.0060 - val_mean_squared_error: 0.0059
Epoch 173/300
929971/929971 [==============================] - 23s 25us/step - loss: 0.0061 - mean_squared_error: 0.0060 - val_loss: 0.0060 - val_mean_squared_error: 0.0059
Epoch 174/300
929971/929971 [==============================] - 23s 25us/step - loss: 0.0061 - mean_squared_error: 0.0060 - val_loss: 0.0060 - val_mean_squared_error: 0.0059
Epoch 175/300
929971/929971 [==============================] - 24s 26us/step - loss: 0.0061 - mean_squared_error: 0.0060 - val_loss: 0.0060 - val_mea

In [ ]:
# Copy data setup file from the clean data folder, and then append network setup.
with open(f'{sub_res}paras.txt', 'a') as file:
    file.write('\n\nThe following is network setup.\n')
    file.write(f'Date and time = {datetime.datetime.now()}\n')
    for n, x in [
        ('Learning rate', hypers['lr']),
        ('L2 regularization alpha', hypers['reg_alpha']),
        ('Output node activation', hypers['outact']),
        ('Nodes per layer', hypers['nodes_per_layer']),
        ('Number of training epochs', hypers['epochs'])
    ]:
        file.write(f'{n} = {x}\n')